In [1]:
import pandas as pd
import json
import re
import os
import sys
import numpy as np
import random
pd.options.display.float_format = '{:,}'.format

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import numpy as npd
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import pandas as pd
import json
import re
import os
import sys
import numpy as np
import random
pd.options.display.float_format = '{:,}'.format

In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
df = pd.read_csv('MD_synopsis.csv')

In [5]:
df

,title,url,year,tconst,isAdult,genres,synopsis
0,Logan,/release/rl3229320705/?ref_=bo_rs_table_4,2017,tt3315342,0,"Action,Drama,Sci-Fi",The theatrical release of Logan was preceded b...
1,The Shack,/release/rl2642183681/?ref_=bo_rs_table_5,2017,tt2872518,0,"Drama,Fantasy","A 13-year-old boy named Mackenzie ""Mack"" Phill..."
2,Before I Fall,/release/rl4001269249/?ref_=bo_rs_table_6,2017,tt1691916,0,"Drama,Fantasy,Mystery",Samantha Kingston wakes on at 6:50 am on Febru...
3,Table 19,/release/rl947815937/?ref_=bo_rs_table_7,2017,tt1412528,0,"Comedy,Drama,Romance",Eloise McGarry attends the wedding of her olde...
4,Kong: Skull Island,/release/rl710837761/?ref_=bo_rs_table_21,2017,tt3731562,0,"Action,Adventure,Fantasy","In the year 1944, somewhere over the South Pac..."
...,...,...,...,...,...,...,...
317,"Where'd You Go, Bernadette",/release/rl2567276033/?ref_=bo_rs_table_47,2019,tt2365580,0,"Comedy,Drama",The film starts with Bernadette Fox rowing a ...
318,Blinded by the Light,/release/rl1409910273/?ref_=bo_rs_table_48,2019,tt8266310,0,"Comedy,Drama,Music","n 1980, young Javed Khan is with his best fri..."
319,Ready or Not,/release/rl285771265/?ref_=bo_rs_table_61,2019,tt7798634,0,"Action,Comedy,Horror","Opening in the year 1989, inside the Le Domas ..."
320,Angel Has Fallen,/release/rl1745257985/?ref_=bo_rs_table_64,2019,tt6189022,0,"Action,Thriller",The film starts with Mike Banning undergoing ...


In [6]:
# synopsis sentence split
synopsis = df[['tconst','synopsis']]

sentences = []
for row in synopsis.itertuples():
    for sentence in sent_tokenize(row[2]):
        sentences.append((row[1], sentence))
new_df = pd.DataFrame(sentences, columns=['ID', 'SENTENCE'])

In [7]:
# create count dummy
new_df['count'] = new_df.groupby(new_df['ID']).transform('count')

# group by
gr = new_df.groupby(['ID','SENTENCE']).count()
ggr = gr.reset_index()

# cumulative sum
ggr['cumsum'] = ggr.groupby('ID')['count'].transform(pd.Series.cumsum)

# sentence count
ggr['total'] = ggr.groupby(ggr['ID']).transform('count').iloc[:,0]

# criterion (50%)
ggr['criterion'] = ggr['total']*0.5

# extract synopsis
df_sentence = ggr[ggr['cumsum'] > ggr['criterion']]

# sentence to document
df_sentence['synopsis'] = df_sentence.groupby(['ID'])['SENTENCE'].transform(lambda x : ' '.join(x))
df_sentence = df_sentence.iloc[:,[0,6]]

# to dataframe
df_doc = df_sentence.drop_duplicates()
df_doc.rename(columns = {'ID':'tconst'}, inplace = True)

C:\Users\User\anaconda3\envs\juunho\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\User\anaconda3\envs\juunho\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [8]:
# filtered movie syopsis (50%)
df.drop(columns = ['synopsis'], inplace = True)

df = df.merge(df_doc, how ='left', on ='tconst',)

In [9]:
df

,title,url,year,tconst,isAdult,genres,synopsis
0,Logan,/release/rl3229320705/?ref_=bo_rs_table_4,2017,tt3315342,0,"Action,Drama,Sci-Fi",Logan runs to intercept them as the Reavers pi...
1,The Shack,/release/rl2642183681/?ref_=bo_rs_table_5,2017,tt2872518,0,"Drama,Fantasy","Mack says he wants to hurt him, and he wants G..."
2,Before I Fall,/release/rl4001269249/?ref_=bo_rs_table_6,2017,tt1691916,0,"Drama,Fantasy,Mystery",Sam tries to apologize and learns from Juliet ...
3,Table 19,/release/rl947815937/?ref_=bo_rs_table_7,2017,tt1412528,0,"Comedy,Drama,Romance",Jo believes the children she has cared for hav...
4,Kong: Skull Island,/release/rl710837761/?ref_=bo_rs_table_21,2017,tt3731562,0,"Action,Adventure,Fantasy",Kong uses a ships propeller attached to a leng...
...,...,...,...,...,...,...,...
317,"Where'd You Go, Bernadette",/release/rl2567276033/?ref_=bo_rs_table_47,2019,tt2365580,0,"Comedy,Drama","She asks Bernadette for a photo, but she silen..."
318,Blinded by the Light,/release/rl1409910273/?ref_=bo_rs_table_48,2019,tt8266310,0,"Comedy,Drama,Music",Javed expresses his desire to go to Manchester...
319,Ready or Not,/release/rl285771265/?ref_=bo_rs_table_61,2019,tt7798634,0,"Action,Comedy,Horror","In response, she punches him in the face and k..."
320,Angel Has Fallen,/release/rl1745257985/?ref_=bo_rs_table_64,2019,tt6189022,0,"Action,Thriller","Mike asserts that he is being framed, and Trum..."


In [10]:
syn_df = df.iloc[:,[6]]

In [11]:
syn_df.to_csv('syn_text.txt', index=False, header=False)

In [12]:
from keybert import KeyBERT

kw_model = KeyBERT()

In [13]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens")
kw_model = KeyBERT(model=sentence_model)

In [14]:
f = open('syn_text.txt', 'r', encoding='UTF-8')
doc = f.read()
keywords = kw_model.extract_keywords(doc)

In [15]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=50)

[('zombies', 0.3269),
 ('zombie', 0.2849),
 ('tornadoes', 0.2818),
 ('thunderstorm', 0.2732),
 ('gunfight', 0.2659),
 ('superman', 0.2551),
 ('deathgrippers', 0.2534),
 ('robbing', 0.2492),
 ('deathstroke', 0.2471),
 ('slaughterhouse', 0.2447),
 ('coyotes', 0.2437),
 ('dropkick', 0.2411),
 ('annihilate', 0.2401),
 ('autobots', 0.2375),
 ('annihilation', 0.2332),
 ('burglarize', 0.2292),
 ('godzillas', 0.2278),
 ('godzilla', 0.2254),
 ('shatterdomes', 0.2233),
 ('fistfight', 0.2231),
 ('werewolf', 0.2213),
 ('hailstorm', 0.221),
 ('tsunami', 0.2208),
 ('wrestling', 0.2205),
 ('carjackers', 0.2196),
 ('autobot', 0.2157),
 ('fleeing', 0.2081),
 ('burglars', 0.2075),
 ('ambushed', 0.2069),
 ('dragonstorm', 0.2012),
 ('apocalypseburg', 0.2003),
 ('apocalypse', 0.1998),
 ('geostorm', 0.196),
 ('terrorist', 0.1952),
 ('sandstorm', 0.1948),
 ('wildfire', 0.1932),
 ('exterminators', 0.1929),
 ('stampede', 0.1917),
 ('kidnapping', 0.1914),
 ('grenades', 0.1901),
 ('deadpool', 0.1893),
 ('gunfire

In [ ]:
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english',use_mmr=True, diversity=0.7, top_n=50)

In [ ]:
keywords

In [ ]:
kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 2), stop_words='english',
                              use_mmr=True, diversity=0.7)

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
# 3개의 단어 묶음인 단어구 추출
n_gram_range = (1, 1)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

In [ ]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
top_n = 50
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

In [ ]:
'''
# 3개의 단어 묶음인 단어구 추출
n_gram_range = (3, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([test_df])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])
'''